# Домашнее задание NLP 5
И пусть твой бот заговорит!

**Цель:**
В этом ДЗ вы создадите телеграм-бота.

<hr>

Описание/Пошаговая инструкция выполнения домашнего задания:
1. Возьмите ОДНО задачи с бенчмарка Russian SuperGLUE (https://russiansuperglue.com/tasks/). Зафайнтьюньте для него модель c сайта huggingface (например, модель RuBERT)

2. Сделайте самбит на Leaderboard https://russiansuperglue.com/leaderboard/2 для оценки качества (в ноутбуке прикрепите скрин сабмита с результатами)
3. Создайте телеграм-бота, в котором будет поднята эта модель. Бот должен вначале писать вводное сообщение с описанием задачи, обрабатывать запрос пользоваля и выводить ответ для заданного пользователем примера. Замечание: пользователь должен писать боту запрос с примером из соотвествующего задания RussianSuperGLUE. под которого зафайнтьюнена модель. Возможны разные варианты реализации: пользователь пишет пример со всеми полями в одном сообщении или в нескольких. Конкретная реализация за вами. Замечание: для проверки можете отправить код с телеграм-ботом и приложить скрины с примерами его работы (в случае если нет возможности поддерживать его в поднятом состоянии)

<hr>

**Критерии оценки:**
<li> Обучена языковая модель за задачах бенчмарка - 4 балла
<li> Успешный сабмит на leaderboard - 2 балла
<li> Обученная модель встроена в телеграм-бота - 4 балла

# 1. Дообучаем ruBert на задачу TERRa из SuperGlue

Датасет есть на huggingface, выгрузим его и немного посмотрим

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

import torch
from transformers import BertTokenizer
import evaluate
from transformers import EvalPrediction

from datasets import load_dataset

dataset = load_dataset("RussianNLP/russian_super_glue", 'terra')

Found cached dataset russian_super_glue (C:/Users/Kate/.cache/huggingface/datasets/RussianNLP___russian_super_glue/terra/0.0.1/d4a95758759a48b32e7a94a8ac7c67046c9111ef4724d7b6a34993cf57eb6048)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'idx', 'label'],
        num_rows: 2616
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'idx', 'label'],
        num_rows: 307
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'idx', 'label'],
        num_rows: 3198
    })
})

In [3]:
dataset['train'][10]

{'premise': 'Музей, расположенный в Королевских воротах, меняет экспозицию. На смену выставке, рассказывающей об истории ворот и их реставрации, придет «Аптека трех королей». Как рассказали в музее, посетители попадут в традиционный интерьер аптеки.',
 'hypothesis': 'Музей закроется навсегда.',
 'idx': 10,
 'label': 1}

Токенизируем данные: будем использовать только основной текст = 'premise' и вывод из него = 'hypothesis'. Не буду использовать padding на этой стадии, выберу DataCollatorWithPadding для выравнивания длины текстов.

Из текста удалим лищние столбцы и переименуем столбец с меткой, так как требует использование Trainer

In [4]:
from transformers import DataCollatorWithPadding

def preprocess_data(examples):
     
    encoding = tokenizer(examples['premise'], examples['hypothesis'])
    return encoding

tokenizer = BertTokenizer.from_pretrained("ai-forever/ruBert-base")

encoded_dataset = dataset.map(preprocess_data, batched = True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding = True)

encoded_dataset = encoded_dataset.remove_columns(['premise', 'hypothesis', 'idx'])
encoded_dataset = encoded_dataset.rename_column("label", "labels")
encoded_dataset.set_format("torch")
encoded_dataset["train"].column_names

Loading cached processed dataset at C:\Users\Kate\.cache\huggingface\datasets\RussianNLP___russian_super_glue\terra\0.0.1\d4a95758759a48b32e7a94a8ac7c67046c9111ef4724d7b6a34993cf57eb6048\cache-73426d8334eb9a7e.arrow
Loading cached processed dataset at C:\Users\Kate\.cache\huggingface\datasets\RussianNLP___russian_super_glue\terra\0.0.1\d4a95758759a48b32e7a94a8ac7c67046c9111ef4724d7b6a34993cf57eb6048\cache-18d08606d892271a.arrow
Loading cached processed dataset at C:\Users\Kate\.cache\huggingface\datasets\RussianNLP___russian_super_glue\terra\0.0.1\d4a95758759a48b32e7a94a8ac7c67046c9111ef4724d7b6a34993cf57eb6048\cache-90ee791c2c7ef912.arrow


['labels', 'input_ids', 'token_type_ids', 'attention_mask']

Добавим метрbre accuracy, так как TERRa ее также использует

In [5]:
ACCURACY = evaluate.load("accuracy")

def compute_metrics(p: EvalPrediction):
    preds = p.predictions
    preds = np.argmax(preds, axis=1)

    acc_result = ACCURACY.compute(predictions=preds, references=p.label_ids)
    result = {"accuracy": acc_result["accuracy"]}

    return result

Запустим обучение и сохраним лучшую модель по accuracy. Эту модель используем для генерации предсказаний в тестовом датасете.

In [6]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained('ai-forever/ruBert-base', num_labels = 2)

training_args = TrainingArguments(

    evaluation_strategy = 'epoch',  
    learning_rate = 2e-05,
    num_train_epochs=3,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=16,              
    weight_decay=0.1,               
    optim="adamw_torch",
    push_to_hub = False,  
    output_dir = "TERRa_model",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    save_strategy="epoch",
    
    

)

trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=encoded_dataset['train'],         
    eval_dataset=encoded_dataset['validation'],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics=compute_metrics
)


trainer.train()

test_dataset = encoded_dataset['test'].remove_columns(['labels'])
predictions = trainer.predict(test_dataset=test_dataset)

Some weights of the model checkpoint at ai-forever/ruBert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not ini

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.611769,0.684039
2,No log,0.555910,0.726384
3,No log,0.646644,0.729642


In [7]:
preds = []
for item in predictions.predictions:
    preds.append(np.argmax(item))

In [8]:
fin = []
for ind, prediction in enumerate(preds):
    res = {}
    res['idx'] = ind
    if prediction == 0:
        res['label'] = "entailment"
    else:
        res['label'] = "not_entailment"
    fin.append(res)
fin[0:10]  

[{'idx': 0, 'label': 'entailment'},
 {'idx': 1, 'label': 'not_entailment'},
 {'idx': 2, 'label': 'entailment'},
 {'idx': 3, 'label': 'not_entailment'},
 {'idx': 4, 'label': 'not_entailment'},
 {'idx': 5, 'label': 'entailment'},
 {'idx': 6, 'label': 'entailment'},
 {'idx': 7, 'label': 'entailment'},
 {'idx': 8, 'label': 'entailment'},
 {'idx': 9, 'label': 'entailment'}]

In [9]:
import json
with open('TERRa.jsonl', 'w') as jsonl_output:
    for entry in fin:
        json.dump(entry, jsonl_output)
        jsonl_output.write('\n')